## Демо по мотивам статьи 
## Unsupervised Machine Translation Using Monolingual Corpora Only

Основные отличия от алгоритма, описанного в статье:

1. Небольшие различия в архитектуре (GRU вместо LSTM), другой оптимизатор, более простая модель дискриминатора
2. Отстутсвие Attention в Decoder
3. Не добавлял шум в автокодировщик

Скачиваем параллельные предложения из multi30K сразу в предобработанном виде.

Основная предобработка: слова и знаки препинания разделены пробелом, и некоторая работа со спецсимволами (типа '&').

In [ ]:
!wget https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/tok/train.lc.norm.tok.fr

In [ ]:
!wget https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/tok/train.lc.norm.tok.en

In [ ]:
!head train.lc.norm.tok.fr

Скачиваем MUSE-вектора, т.е. вектора для слов на разных языках, выровненные таким образом, чтобы косинусное расстояние между схожими словами на разных языках было невелико. 

In [ ]:
!wget https://s3.amazonaws.com/arrival/embeddings/wiki.multi.en.vec

In [ ]:
!wget https://s3.amazonaws.com/arrival/embeddings/wiki.multi.fr.vec

Импорт основных библиотек

In [ ]:
import io
import numpy as np

import unicodedata
import string
import re
import random
import codecs
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print device

Из обучалки torch Seq2Seq: удаляем всё, кроме латинских букв и знаков препинания. 
    
Диакритические знаки (черточки над буквами) также удаляем.

In [ ]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s.strip()

In [ ]:
normalizeString(u"bonjour  je suis élève de l'institut'")

Функция загрузки векторов MUSE из файла.
Словари векторов содержат много мусорных слов (типа хэштегов), поэтому используем следующую логику:
    
1. Предобрабатываем слова, удаляем нелатинские символы
2. Если после предобработки в словаре встретился дубликат:
        a. Если текущее слово не изменилось после предобработки (т.е. оно скорее всего "хорошее"), то заменяем для данного слова в словаре вектор на текущий
        b. Иначе пропускаем и идем дальше

In [ ]:
def load_vec(emb_path):
    vectors = []
    word2id = {}
    with io.open(emb_path, 'r', encoding='utf-8', newline='\n', errors='ignore') as f:
        next(f)
        for i, line in enumerate(f):
            orig_word, vect = line.rstrip().split(' ', 1)
            
            word = normalizeString(orig_word)
            vect = np.fromstring(vect, sep=' ')
            if word in word2id:
                print u'word found twice: {0} ({1})'.format(word, orig_word)
                if orig_word==word:
                    id = word2id[word]
                    vectors[id] = vect
                    print 'rewriting'
                    continue
                else:
                    continue
            vectors.append(vect)
            word2id[word] = len(word2id)
            
    id2word = {v: k for k, v in word2id.items()}
    embeddings = np.vstack(vectors)
    return embeddings, id2word, word2id

In [ ]:
en_embedding_tuple = load_vec('./wiki.multi.en.vec')

In [ ]:
fr_embedding_tuple = load_vec('./wiki.multi.fr.vec')

Функция из MUSE: находим ближайшие вектора из другого языка. 

Посмотрим, насколько хорошо работает выравнивание.

In [ ]:
def get_nn(word, src_emb, src_id2word, tgt_emb, tgt_id2word, K=5):
    print("Nearest neighbors of \"%s\":" % word)
    word2id = {v: k for k, v in src_id2word.items()}
    word_emb = src_emb[word2id[word]]
    scores = (tgt_emb / np.linalg.norm(tgt_emb, 2, 1)[:, None]).dot(word_emb / np.linalg.norm(word_emb))
    k_best = scores.argsort()[-K:][::-1]
    for i, idx in enumerate(k_best):
        print('%.4f - %s' % (scores[idx], tgt_id2word[idx]))

In [ ]:
for word in ['cat','dog','human','student','computer']:
    get_nn(word, en_embedding_tuple[0], en_embedding_tuple[1], fr_embedding_tuple[0], fr_embedding_tuple[1], K=3)

Класс Lang отвечает за обработку языка.

SOS_token --- идентификатор начала предложения.

EOS_token --- идентификатор конца предложения.

In [ ]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name, embedding_tuple):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = [ "SOS", "EOS"]
        self.embedding_tuple = embedding_tuple    
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:   
            if word in self.embedding_tuple[2]:
                self.word2index[word] = self.n_words
                self.word2count[word] = 1
                self.index2word.append(word)
                self.n_words += 1
        else:
            self.word2count[word] += 1
            
    def get_matrix(self):
        """
        Получаем матрицу слово -> вектор для всех слов, которые встретились в тексте.
        Вектор для начала предложения заменяем нулевым, 
        для конца предложения --- единичным (можно заменить на случайный вектор).
        """
        dim = self.embedding_tuple[0].shape[1]
        matrix = np.zeros((self.n_words, dim))        
        matrix[0] = np.zeros(dim)
        matrix[1] = np.ones(dim)
        for id, word in enumerate(self.index2word[2:]):
            id = id+2
            word_id = self.embedding_tuple[2][word]
            vector = self.embedding_tuple[0][word_id]
            matrix[id] = vector
        return matrix

In [ ]:
# Предложение представляется как набор идентификаторов слов. 
# Чтобы поместить несколько предложений в одну матрицу, нужно дополнить каждое предложение токенами конца предложения.
def pad_seq(seq, length):
    
    seq += [EOS_token for i in range(length - len(seq))]
    return seq

Функции-утилиты для считывания текста.

В дальнейшем ограничимся только предложениями длины <= 10 для экономии памяти.

In [ ]:
def readLangs(lang1, lang2, emb1, emb2,  prefix):
    print("Reading lines...")

    # Read the file and split into lines
    lines1 = codecs.open(prefix+lang1, encoding='utf-8').\
        read().strip().split('\n')
    lines2 = codecs.open(prefix+lang2, encoding='utf-8').\
        read().strip().split('\n')
    # Split every line into pairs and normalize
    lines1 = [normalizeString(s) for s in lines1]
    lines2 = [normalizeString(s) for s in lines2]
   
    input_lang = Lang(lang1, emb1)
    output_lang = Lang(lang2, emb2)

    return input_lang, output_lang, lines1, lines2

In [ ]:
MAX_LENGTH = 10


def filter_line(line):
    return len(line.split(' ')) < MAX_LENGTH
    
def filter_lines(lines):
    return [line for line in lines if filter_line(line)]

In [ ]:
def prepareData(lang1, lang2, emb1, emb2,  prefix):
    """
    Возвращает два объекта-языка, предложения для обучения
    и пары параллельных предложений для промежуточной валидации результата
    """
    input_lang, output_lang, lines1, lines2 = readLangs(lang1, lang2,  emb1, emb2, prefix)
    print("Read %s sentence pairs" % len(lines1))
    pairs = [(l1, l2) for l1, l2 in zip(lines1, lines2) if filter_line(l1) and filter_line(l2)]
    
    # по условиями эксперимента у нас нет параллельных предложений. Для чистоты эксперимента  перемешаем их.
    np.random.shuffle(lines1)
    np.random.shuffle(lines2)

    lines1 = filter_lines(lines1)
    lines2 = filter_lines(lines2)
    
    min_lines = min(len(lines1), len(lines2))
    lines1, lines2 = lines1[:min_lines], lines2[:min_lines]
    
    print("Trimmed to %s sentence pairs" % min_lines)
    print("Counting words...")
    for l1, l2 in zip(lines1, lines2):
        input_lang.addSentence(l1)
        output_lang.addSentence(l2)
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, lines1, lines2,  pairs


input_lang, output_lang, lines1, lines2, pairs = prepareData('fr', 'en', fr_embedding_tuple, en_embedding_tuple, 
                                             'train.lc.norm.tok.')
# в качестве щашумленного перевода предложений пока просто возьмем непараллельные пары
tr_lines1,tr_lines2 = lines2[:], lines1[:]

Функции кодирования предложений в последовательность идентификаторов слов

In [ ]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ') if word in lang.word2index]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)


# функция возвращает матрицы случайных предложений и их зашумленных версий в pytorch-формате.
# batch_size --- количество предложений.
def random_batch(batch_size):
    input_seqs = []
    target_seqs = []
    tr_input_seqs = []
    tr_target_seqs = []
    
    # Choose random pairs
    for i in range(batch_size):
        id1 = random.choice(xrange(len(lines1)))
        id2 = random.choice(xrange(len(lines2)))
        line1 = lines1[id1]
        line2 = lines2[id2]
        tr_line1 = tr_lines1[id1]
        tr_line2 = tr_lines2[id2]
        
        
        input_seqs.append(indexesFromSentence(input_lang, line1))
        target_seqs.append(indexesFromSentence(output_lang, line2))
        
        tr_input_seqs.append(indexesFromSentence(output_lang, tr_line1))
        tr_target_seqs.append(indexesFromSentence(input_lang, tr_line2))
        
        
    input_length = max([len(s) for s in input_seqs])
    target_length = max([len(s) for s in target_seqs])
    tr_input_length = max([len(s) for s in tr_input_seqs])
    tr_target_length = max([len(s) for s in tr_target_seqs])
    
    
    # For input and target sequences, get array of lengths and pad with 0s to max length    
    input_padded = [pad_seq(s, input_length) for s in input_seqs]    
    target_padded = [pad_seq(s, target_length) for s in target_seqs]
    
    tr_input_padded = [pad_seq(s, tr_input_length) for s in tr_input_seqs]    
    tr_target_padded = [pad_seq(s, tr_target_length) for s in tr_target_seqs]

    
    # Turn padded arrays into (batch_size x max_len) tensors, transpose into (max_len x batch_size)
    input_var = torch.tensor(input_padded, dtype=torch.long,device=device).transpose(0, 1)
    target_var = torch.tensor(target_padded, dtype=torch.long,device=device).transpose(0, 1)
    tr_input_var = torch.tensor(tr_input_padded, dtype=torch.long,device=device).transpose(0, 1)
    tr_target_var = torch.tensor(tr_target_padded, dtype=torch.long,device=device).transpose(0, 1)
    
    
    return input_var, target_var, tr_input_var, tr_target_var

In [ ]:
batch = random_batch(30)

Посмотрим матрицы для слов из каждого языка для использования в Seq2Seq

In [ ]:
fr_matrix = torch.FloatTensor(input_lang.get_matrix())


In [ ]:
en_matrix = torch.FloatTensor(output_lang.get_matrix())


Простой класс однослойной нейросети с 1000 нейронов на скрытом слое, будет использоваться как дискриминатор.

Входная размерность --- 300, совпадает с размерностью слов и размерностью скрытого пространства Seq2Seq,

In [ ]:
class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        self.fc1 = nn.Linear(300,1000)
        self.fc2 = nn.Linear(1000, 1)
        
    def forward(self, x):
        x = torch.nn.functional.relu(self.fc1(x))
        y = torch.nn.functional.sigmoid(self.fc2(x))
        
        return y

класс Encoder.

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, matrix, gru = None):
        """
        gru -- если не None, берет готовую модель gru и использует для своего языка.
        Параметр требуется, чтобы использовать одну и ту же GRU-модель для двух энкодеров с разных языков.
        """
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        
        
        self.embedding = nn.Embedding.from_pretrained(matrix)
        self.embedding.requires_grad = False
        
        if not gru:
            self.gru = nn.GRU(hidden_size, hidden_size)
        else:
            self.gru = gru
            

    def forward(self, input, batch_size, hidden):
        embedded = self.embedding(input).view(1, batch_size, self.hidden_size)
        output = embedded
        
        output, hidden = self.gru(output, hidden)
        return output, hidden
    

    def initHidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size, device=device)

Decoder. 

Важно: в отличие от статьи, я не использовал Attention. 

Attention можно взять из официальной обучалки pytorch seq2seq, но он там работает с одним предложением за одну операцию.
Лучше погуглить "seq2seq pytorch batch"

In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, matrix, gru = None):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding.from_pretrained(matrix)
        self.embedding.requires_grad = False
        if not gru:
            self.gru = nn.GRU(hidden_size, hidden_size)
        else:
            self.gru = gru
            
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden, batch_size):
        output = self.embedding(input).view(1, batch_size, self.hidden_size)
        
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden
    
    def initHidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size, device=device)

Функции для проведения одной итерации оптимизации.

teacher_forcing_ratio отвечает вероятность, что в качестве слов при декоде мы будем получать подсказку, а не будем использовать те слова, что были раскодированы декодером самостоятельно на предыдущих шагах. 

In [ ]:
teacher_forcing_ratio = 0.5

def encode_decode(input_tensor,  encoder, decoder, target_tensor=None):
    # раскодирует и декодирует.
    # input_tensor, target_tensor --- матрицы размером
    #    <Количество предложений в батче> * <Максимальная длина предложения в батче>
    # возвращает последовательность идентификаторов слов от декодера и скрытый вектор от энкодера
    batch_size = input_tensor.size(1)
    encoder_hidden = encoder.initHidden(batch_size)
    
    input_length = input_tensor.size(0)
    if target_tensor is None:
        target_tensor = input_tensor
    
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(input_length, batch_size, encoder.hidden_size, device=device)

    for ei in range(input_length):
    
        encoder_output, encoder_hidden = encoder(input_tensor[ei], batch_size, encoder_hidden)
        encoder_outputs[ei] = encoder_output[0]
    
    
    
    decoder_input = torch.tensor([[SOS_token]*batch_size], device=device)
    decoder_hidden = encoder_hidden
    
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    outputs = []
    for di in range(target_length):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, batch_size)        
        outputs.append(decoder_output)        
        
        if use_teacher_forcing:
            decoder_input = target_tensor[di]  # Teacher forcing            
        else:
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input
            #if decoder_input == EOS_token:
            #    break
    return outputs,  encoder_hidden


def train(source_tensor, target_tensor, translated_source_tensor, translated_target_tensor, 
          encoder_source, encoder_target,
          decoder_source, decoder_target, discriminator,  
          optimizer,   discriminator_optimizer, 
          criterion, cross_entropy, ae_coef = 1.0, translate_coef = 0.0, disc_coef = 1.0,   max_length=MAX_LENGTH):
    """
    Одна итерация оптимизации
    source_tensor --- матрица предложений первого языка
    target_tensor --- матрица предложений второго языка
    translated_source_tensor --- зашумленный перевод первого языка
    translated_target_tensor --- зашумленный перевод второго языка
    encoder_target, decoder_source, encoder_target, decoder_target --- seq2seq кодировщики
    discriminator --- дискриминаторная сеть
    optimizer, discriminator_optimizer --- оптимизаторы Seq2Seq и дискриминатора
    criterion --- функция ошибки для Seq2Seq. Принимает на вход предложения и матрицу ответа от декодера
    cross_entropy --- функция ошибки для дискриминатора
    """
    
    batch_size = source_tensor.size(1)
    
    # автокодировщики
    source_to_source, source_hidden = encode_decode(source_tensor, encoder_source, decoder_source )
    target_to_target, target_hidden = encode_decode(target_tensor, encoder_target, decoder_target)
    
    # зануляем градиент от ошибки Seq2Seq
    optimizer.zero_grad()
    
    
    loss = 0
    
    # считаем ошибку от первого автокодировщика
    for di in range(len(source_to_source)):
        decoder_output = source_to_source[di]
        loss += criterion(decoder_output, source_tensor[di])*ae_coef
    
    
    #считаем ошибку от второго автокодировщика
    for di in range(len(target_to_target)):
        decoder_output = target_to_target[di]
        loss += criterion(decoder_output, target_tensor[di])*ae_coef
    
    # будем считать, что объекты из ПЕРВОГО языка принадлежат  классу "1", 
    # объекты ВТОРОГО языка --- классу "0"
    classes = torch.tensor(np.array([1.0]*batch_size + [0.0]*batch_size), dtype=torch.float, device=device)
    
    # Смотрим, что предсказал дискриминатор
    source_hidden_predict = discriminator(torch.stack([target_hidden, source_hidden]))
    # На данном этапе мы хотим обмануть дискриминатор, поэтому будем минимизировать долю правильных ответов,
    # т.е. минимизировать ошибку между ответами дискриминатора и НЕПРАВИЛЬНЫМИ классами.
    loss += cross_entropy(source_hidden_predict, classes)*disc_coef
    
    # аналогично автокодирощикам, считаем ошибку на зашумленном переводе
    if translate_coef > 0.0:
        translated_source_to_source, _ = encode_decode(translated_source_tensor, encoder_target, decoder_source, target_tensor=source_tensor)
        translated_target_to_target, _ = encode_decode(translated_target_tensor, encoder_source, decoder_target, target_tensor=target_tensor)
        
        for di in range(min(len(translated_source_to_source), len(source_tensor))):
            decoder_output = translated_source_to_source[di]
            loss += criterion(decoder_output, source_tensor[di])*ae_coef

        for di in range(min(len(translated_target_to_target), len(target_tensor))):
            decoder_output = translated_target_to_target[di]
            loss += criterion(decoder_output, target_tensor[di])*ae_coef



    # подсчет градиента от ошибки
    loss.backward()

    # запуск оптимизации в сторону антиградиента
    optimizer.step()
    
    # информация для отладки
    avg_len = (len(target_tensor)+len(source_tensor))/2
    
    # обнуляем градиент и ошибку для дискриминатора
    d_loss = 0     
    discriminator_optimizer.zero_grad()
    
    _, source_hidden = encode_decode(source_tensor, encoder_source, decoder_source)
    _, target_hidden = encode_decode(target_tensor, encoder_target, decoder_target)
    
    
    # теперь минимизируем ошибку между предсказанным и правильным классами
    source_hidden_predict = discriminator(torch.stack([source_hidden, target_hidden]))    
    d_loss += cross_entropy(source_hidden_predict, classes)

    d_loss.backward()

    discriminator_optimizer.step()
    
              
    
    return loss.item() / avg_len, d_loss.item()

In [ ]:
import time

def trainIters(encoder_source,encoder_target,  decoder_source,  decoder_target,discriminator, 
               n_iters, print_every=1000,  learning_rate=0.001):
    # глобальная процедура оптимизации 
    
    
    global tr_lines1, tr_lines2
    
    optimizer = optim.Adam(list(encoder_source.parameters()) +  list(decoder_source.parameters())+\
                            list(encoder_target.parameters())+  list(decoder_target.parameters()),
                           lr=learning_rate)
    
    optimizer2 = optim.Adam(discriminator.parameters(),
                           lr=learning_rate)
    
    
    criterion = nn.NLLLoss()
    criterion2 = nn.BCELoss()
    print_loss_total = [0, 0]

    for iter in range(1, n_iters + 1):    
        batch = random_batch(25)
        
        # уровень доверия к зашумленному переводу будет увеличиваться в процессе оптимизации. 
        # поскольку в первое время мы не обладаем никаким переводом, то изначально коэффициент будет нулевым.
        if iter < print_every:
            tr_coef = 0.0
        else:
            tr_coef = 1.0/n_iters
        loss = train(batch[0], batch[1], batch[2], batch[3],  encoder_source,encoder_target,  decoder_source,  decoder_target,
                     discriminator, optimizer, optimizer2, criterion, criterion2, translate_coef = tr_coef)
        print_loss_total[0] += loss[0]
        print_loss_total[1] += loss[1]
        

        if iter % print_every == 0:
            # выводи примеры перевода, среднюю ошибку и делаем новый шумный перевод, 
            # каждую 'print_every' итерацию.
            print_loss_avg0 = print_loss_total[0] / print_every
            print_loss_avg1 = print_loss_total[1] / print_every
            print_loss_total = [0,0]
            print iter, print_loss_avg0, print_loss_avg1
            print '_'*10
            evaluateRandomly(encoder_source, decoder_target,n=3)
            print '_'*10
            evaluateRandomly(encoder_source, decoder_source,langid=0, n=3)
            print '_'*10
            evaluateRandomly(encoder_target, decoder_target, langid=1, n=3)
            
            tr_lines1, tr_lines2 = make_translation()

In [ ]:
def make_translation():
    """
    Построение зашуменного перевода. 
    Можно существенно ускорить, если переводить батчем.
    """
    max_length = MAX_LENGTH
    tr_lines1, tr_lines2 = [],[]
    id = 0
    for line in lines1:
        id+=1
        if id%1000 == 0:
            print 'translating source', id
        input_tensor = tensorFromSentence(input_lang, line)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder_source.initHidden(1)

        encoder_outputs = torch.zeros(max_length, encoder_source.hidden_size, device=device)

        
        for ei in range(min(MAX_LENGTH, input_length)):
            encoder_output, encoder_hidden = encoder_source(input_tensor[ei],1,
                                                     encoder_hidden)
            
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        
        for di in range(max_length):
            decoder_output, decoder_hidden = decoder_target(
                decoder_input, decoder_hidden, 1)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:                
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()
        tr_lines1.append(u' '.join(decoded_words))
    
    # можно устранить дублирование кода, не дошли руки, прим. Олег
    id  = 0 
    for line in lines2:
        id+=1
        if id%1000 == 0:
            print 'translation target', id
        input_tensor = tensorFromSentence(output_lang, line)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder_target.initHidden(1)

        encoder_outputs = torch.zeros(max_length, encoder_target.hidden_size, device=device)

        
        for ei in range(min(MAX_LENGTH, input_length)):
            encoder_output, encoder_hidden = encoder_target(input_tensor[ei],1,
                                                     encoder_hidden)
            
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        
        for di in range(max_length):
            decoder_output, decoder_hidden = decoder_source(
                decoder_input, decoder_hidden, 1)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                
                break
            else:
                decoded_words.append(input_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()
        tr_lines2.append(u' '.join(decoded_words))
        
    
    return tr_lines1, tr_lines2
#tr_lines1, tr_lines2 = make_translation()

In [ ]:
def evaluate(encoder, decoder, sentence, encoded_lang,  decoder_lang, max_length=MAX_LENGTH):
    """
    Процедура промежуточной валидации перевода
    """
    with torch.no_grad():
        input_tensor = tensorFromSentence(encoded_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden(1)

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(min(MAX_LENGTH, input_length)):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],1,
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        
        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, 1)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(decoder_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words
    
def evaluateRandomly(encoder, decoder, langid = -1,  n=10):
    """
    Берем n предложений и смотрим качество на них.
    Если langid == -1 --- смотрим качество перевода.
    Если langid == 0 или == 1, смотирм качество восстановления автокодировщиком.
    """
    for i in range(n):
        if langid == -1:
            id0 = 0
            id1 = 1
            enc_lang= input_lang
            dec_lang = output_lang
        else:            
            id0 = langid
            if langid == 0:
                enc_lang = input_lang
                dec_lang = input_lang
            else:
                enc_lang = output_lang
                dec_lang = output_lang
        if langid==-1:
                
            pair = random.choice(pairs)
        else:
            pair = [random.choice(lines1), random.choice(lines2)]
        print   '>', pair[id0]
        if langid==-1:
            print '=', pair[id1]
        
            
        output_words = evaluate(encoder, decoder, pair[id0], enc_lang, dec_lang)
        output_sentence = ' '.join(output_words)
        print '<', output_sentence, '\n'
        
    

Создание сетей и запуск обучения.

Для информации ниже: пример вывода валидации на последних 10000 итераций.

Видно, что перевод работает, хотя и не всегда корректно.

Зато почти идеально выполняется восстановление предложений.

Добавление шума в автокодировщик (как в статье) исправит положение.

In [ ]:

hidden_size = 300

encoder_source = EncoderRNN(input_lang.n_words, hidden_size, fr_matrix).to(device)
encoder_target = EncoderRNN(output_lang.n_words, hidden_size, en_matrix, gru = encoder_source.gru).to(device)

decoder_source = DecoderRNN(hidden_size, input_lang.n_words, fr_matrix).to(device)
decoder_target = DecoderRNN(hidden_size, output_lang.n_words, en_matrix, gru = decoder_source.gru).to(device)

disc = Net1()
disc.cuda()


In [ ]:
trainIters(encoder_source,encoder_target,  decoder_source,  decoder_target, disc, 100000, print_every=5000)

Сохраняем модели

In [ ]:
torch.save(encoder_source.gru.state_dict(), 'monolingual_seq2seq_fr_en_enc')
torch.save(decoder_source.gru.state_dict(), 'monolingual_seq2seq_fr_en_dec')